### IMPORT OF NEED LIBRARIES

In [1]:
import requests
import pandas as pd
import json
from datetime import datetime

In [2]:
#list of cities to be studied
list_cities = ["Mont Saint Michel", "St Malo","Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille",
               "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon",
               "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas" ,"Cassis", "Marseille",
               "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer",
               "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

### GPS COORDINATES SCRAPPING

In [3]:
#Iteration on the studied list of cities 
#in order to get back their gps coordinates from the API https://nominatim.org/ 

list_gps = []

for element in list_cities:
    r_gps = requests.get("https://nominatim.openstreetmap.org/search?q={}&format=json".format(element)) #API request
    dic_gps = r_gps.json()
    lat = float(dic_gps[0]["lat"])
    lon = float(dic_gps[0]["lon"])
    list_gps.append([lat,lon]) #list containing latitude and longitude for each city

### WEATHER SCRAPPING AND ADDING TO DATAFRAME

In [4]:
#The Dataframe will contain the gps coordinates as well as weather information for next 7 days for each city

columns = ["city_name", "city_id", "latitude", "longitude", "day_month", "weather_desc",
         "prob_precipitation", "daily_temperature"]

dataset = pd.DataFrame(columns = columns)
dataset

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature


In [5]:
key_api = "759b23dd77cc2ec29a8c0fcd794b65e0"

for j in range(len(list_cities)):
    
    city = list_cities[j]
    city_id = j
    lat = list_gps[j][0]
    lon = list_gps[j][1]
    
    #requesting on the API https://openweathermap.org/appid 
    r_weather = requests.get(
    "https://api.openweathermap.org/data/2.5/onecall?lat={}&lon={}&units={}&exclude=minutely,hourly&appid={}"\
        .format(lat, lon, 'metric', key_api)) 
    
    
    dic_weather = r_weather.json()
 
    dictionnary={'city_name': city,
                 'city_id':city_id,
                 'latitude':lat,
                 'longitude':lon}
    
    for i in range(1,8):
        timestamp = dic_weather["daily"][i]["dt"]
        date = datetime.fromtimestamp(timestamp)
        dictionnary['day_month'] = date.strftime("%d/%m")
        dictionnary['weather_desc'] = dic_weather["daily"][i]["weather"][0]["description"]
        dictionnary['daily_temperature'] = dic_weather["daily"][i]['temp']['day']
        dictionnary['prob_precipitation'] = float(dic_weather["daily"][i]["pop"])
        dataset = dataset.append(dictionnary, ignore_index = True)

In [6]:
dataset.head(10)

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature
0,Mont Saint Michel,0,48.635856,-1.512610,25/01,light rain,0.69,5.40
1,Mont Saint Michel,0,48.635856,-1.512610,26/01,moderate rain,1.00,4.96
2,Mont Saint Michel,0,48.635856,-1.512610,27/01,light rain,1.00,10.38
3,Mont Saint Michel,0,48.635856,-1.512610,28/01,moderate rain,1.00,12.54
4,Mont Saint Michel,0,48.635856,-1.512610,29/01,moderate rain,1.00,11.31
5,Mont Saint Michel,0,48.635856,-1.512610,30/01,heavy intensity rain,1.00,12.21
6,Mont Saint Michel,0,48.635856,-1.512610,31/01,light rain,0.76,10.83
7,St Malo,1,48.645453,-2.015418,25/01,light rain,0.71,6.03
8,St Malo,1,48.645453,-2.015418,26/01,light rain,0.96,5.62
9,St Malo,1,48.645453,-2.015418,27/01,light rain,1.00,10.18


In [7]:
#Computing the average temperature of the next 7 days for the different cities

mean_temp = dataset.groupby("city_name")["daily_temperature"]\
                .mean()\
                .reset_index()

mean_temp.rename(columns = {"city_name": "city_name", 
                          "daily_temperature": "average_temperature"}, 
                 inplace = True)
mean_temp

,city_name,average_temperature
0,Aigues Mortes,11.731429
1,Aix en Provence,10.550000
2,Amiens,7.411429
3,Annecy,4.307143
4,Ariege,6.034286
5,Avignon,10.828571
6,Bayeux,8.734286
7,Bayonne,12.797143
8,Besancon,6.058571
9,Biarritz,12.507143


In [8]:
#Adding the average temperature in the main dataset

dataset = dataset.merge(mean_temp, on = "city_name", how = "inner")
dataset

,city_name,city_id,latitude,longitude,day_month,weather_desc,prob_precipitation,daily_temperature,average_temperature
0,Mont Saint Michel,0,48.635856,-1.512610,25/01,light rain,0.69,5.40,9.661429
1,Mont Saint Michel,0,48.635856,-1.512610,26/01,moderate rain,1.00,4.96,9.661429
2,Mont Saint Michel,0,48.635856,-1.512610,27/01,light rain,1.00,10.38,9.661429
3,Mont Saint Michel,0,48.635856,-1.512610,28/01,moderate rain,1.00,12.54,9.661429
4,Mont Saint Michel,0,48.635856,-1.512610,29/01,moderate rain,1.00,11.31,9.661429
...,...,...,...,...,...,...,...,...,...
240,La Rochelle,34,46.159113,-1.152043,27/01,light rain,0.55,11.28,9.927143
241,La Rochelle,34,46.159113,-1.152043,28/01,light rain,0.86,12.43,9.927143
242,La Rochelle,34,46.159113,-1.152043,29/01,light rain,1.00,11.04,9.927143
243,La Rochelle,34,46.159113,-1.152043,30/01,light rain,1.00,13.31,9.927143


In [11]:
#Saving the weather information in a CSV file
dataset.to_csv('res/35 cities - 7 days weatherforecast.csv', index = False)